In [1]:
import os
import sys
sys.path.append(os.path.dirname((os.path.abspath(''))))

from src.pipeline import Pipeline
from src.data.preprocessing import removal, tokenization, stemming

[nltk_data] Downloading package punkt to /Users/tim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/tim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
pipeline = Pipeline()

In [3]:
pipeline.setup()

In [24]:
features_test.head()

,qID,pID
0,1108939,945864
1,1108939,2776470
2,1108939,1897608
3,1108939,2707038
4,1108939,5712720


In [23]:
embeddings = load('data/embeddings/tfidf_collection_embeddings.pkl')
embeddings_queries = load('data/embeddings/tfidf_query_embeddings.pkl')

embeddings_queries[pipeline.queries[pipeline.queries['qID'] == 914462].index]
#embeddings[pipeline.collection[pipeline.collection['pID'] == 3852073].index]

<1x63171 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [ ]:
from src.features.generator import create_all
create_all(features_test, pipeline.collection, pipeline.queries_test)

/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
 46%|█████████████████████████████████████████████████████████▌                                                                   | 460240/1000000 [13:54<17:39, 509.56it/s]

In [8]:
pipeline.queries_test

,qID,Query,preprocessed
0,1108939,what slows down the flow of blood,"[slow, flow, blood]"
1,1112389,"what is the county for grand rapids, mn","[counti, grand, rapid, mn]"
2,792752,what is ruclip,[ruclip]
3,1119729,what do you do when you have a nosebleed from ...,"[noseble, nose]"
4,1105095,where is sugar lake lodge located,"[sugar, lake, lodg, locat]"
...,...,...,...
195,146187,difference between a mcdouble and a double che...,"[differ, mcdoubl, doubl, cheeseburg]"
196,634428,what does chs stand for?,"[ch, stand]"
197,1121986,what are the effects of having low blood sugar,"[effect, low, blood, sugar]"
198,321441,how much is a us postal stamp cost,"[much, us, postal, stamp, cost]"


In [4]:
pipeline.features

,qID,pID,y
0,111791,5158606,1
4,914462,3852073,1
5,201329,3704115,1
6,212165,5773592,1
14,32031,6243218,1
...,...,...,...
3190,844263,2591163,0
3191,844263,4064917,0
3192,844263,2111580,0
3193,844263,6292238,0


In [5]:
pipeline.qrels

,qID,pID,feedback
0,19335,1017759,0
1,19335,1082489,0
2,19335,109063,0
3,19335,1160863,0
4,19335,1160871,0
...,...,...,...
9255,1133167,8839920,2
9256,1133167,8839922,2
9257,1133167,944810,0
9258,1133167,949411,0


In [4]:
pipeline.preprocess()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 665.97it/s]


In [6]:
pipeline.create_BM25_features()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4244/4244 [00:04<00:00, 1057.65it/s]


In [7]:
pipeline.collection

,pID,Passage,preprocessed
0,945864,Notice how their voice goes up and down very d...,"[notic, voic, goe, deliber, know, well, notic,..."
1,2776470,Definition of inter-. 1 1 : between : among :...,"[definit, inter, 1, 1, among, midst, intercrop..."
2,1897608,"Literary consonance. From Wikipedia, the free ...","[literari, conson, wikipedia, free, encycloped..."
3,2707038,"ABOUT MARINELLO. For more than 100 years, Mari...","[marinello, 100, year, marinello, educ, studen..."
4,5712720,3. Ginger. Due to its anti-inflammatory proper...,"[3, ginger, due, antiinflammatori, properti, g..."
...,...,...,...
24995,8343268,How Much Does Assisted Living Cost? The answer...,"[much, assist, live, cost, answer, depend, com..."
24996,5169339,Posted on 03/29/2017. The Miss Marlboro County...,"[post, 03292017, miss, marlboro, counti, high,..."
24997,1313435,"Maryellen, sometimes written as Mary Ellen, is...","[maryellen, sometim, written, mari, ellen, eng..."
24998,2292614,From Texas flooding to now drought and fire we...,"[texa, flood, drought, fire, weather, war, pot..."


In [8]:
pipeline.create_tfidf_embeddings()

/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 8306.62it/s]


In [9]:
pipeline.create_tfidf_feature()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6390/6390 [00:11<00:00, 552.89it/s]


In [10]:
pipeline.create_jaccard_feature()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6390/6390 [00:05<00:00, 1247.62it/s]


In [11]:
pipeline.create_sentence_features()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6390/6390 [00:00<00:00, 57632.53it/s]


In [12]:
pipeline.create_interpretation_features()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6390/6390 [00:05<00:00, 1187.42it/s]


In [13]:
pipeline.create_POS_features()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6390/6390 [00:08<00:00, 755.68it/s]


In [14]:
pipeline.queries_test

,qID,Query,preprocessed,tfidf
0,1108939,what slows down the flow of blood,"[slow, flow, blood]","{'slow': 0.65816726040214, 'flow': 0.591294616..."
1,1112389,"what is the county for grand rapids, mn","[counti, grand, rapid, mn]","{'counti': 0.35972012398720704, 'grand': 0.508..."
2,792752,what is ruclip,[ruclip],{'ruclip': 0.0}
3,1119729,what do you do when you have a nosebleed from ...,"[noseble, nose]","{'noseble': 0.8307258285426083, 'nose': 0.5566..."
4,1105095,where is sugar lake lodge located,"[sugar, lake, lodg, locat]","{'sugar': 0.4661892752744652, 'lake': 0.509815..."
...,...,...,...,...
195,146187,difference between a mcdouble and a double che...,"[differ, mcdoubl, doubl, cheeseburg]","{'differ': 0.5368559628243917, 'mcdoubl': 0.0,..."
196,634428,what does chs stand for?,"[ch, stand]","{'ch': 0.8476330205893083, 'stand': 0.53058294..."
197,1121986,what are the effects of having low blood sugar,"[effect, low, blood, sugar]","{'effect': 0.4606853125805391, 'low': 0.505444..."
198,321441,how much is a us postal stamp cost,"[much, us, postal, stamp, cost]","{'much': 0.3323141330856561, 'us': 0.316834505..."


In [7]:
pipeline.features

,qID,pID,y,bm25
0,111791,5158606,1,-31.065276
4,914462,3852073,1,-26.047702
5,201329,3704115,1,-18.131038
6,212165,5773592,1,-16.103601
14,32031,6243218,1,-24.332336
...,...,...,...,...
3190,844263,2591163,0,-22.982374
3191,844263,4064917,0,-18.607365
3192,844263,2111580,0,-22.154547
3193,844263,6292238,0,-28.027324


In [8]:
import pandas as pd
import numpy as np

pipeline.collection.preprocessed.apply(lambda passage: np.isin('wikipedia', passage)).value_counts()[1]

48

In [33]:
pipeline.collection.preprocessed.size

25000

In [32]:
len(pipeline.collection.preprocessed[2])

71

In [10]:
pipeline.collection.preprocessed.progress_apply(lambda passage: passage.size).mean()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25000/25000 [00:00<00:00, 61765.92it/s]


33.71828

In [9]:
from src.features.generator import create_tfidf_embeddings, create_all
from src.utils.utils import load 

create_all(features_test, pipeline.collection, pipeline.queries_test, load('models/tfidf.pkl'))

TypeError: transform() takes 2 positional arguments but 3 were given

In [30]:
create_all(features_test, pipeline.collection, pipeline.queries_test)

/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 8934.22it/s]


IsADirectoryError: [Errno 21] Is a directory: 'data/embeddings'

In [11]:
pipeline.queries_test

,qID,Query,preprocessed
0,1108939,what slows down the flow of blood,"[slow, flow, blood]"
1,1112389,"what is the county for grand rapids, mn","[counti, grand, rapid, mn]"
2,792752,what is ruclip,[ruclip]
3,1119729,what do you do when you have a nosebleed from ...,"[noseble, nose]"
4,1105095,where is sugar lake lodge located,"[sugar, lake, lodg, locat]"
...,...,...,...
195,146187,difference between a mcdouble and a double che...,"[differ, mcdoubl, doubl, cheeseburg]"
196,634428,what does chs stand for?,"[ch, stand]"
197,1121986,what are the effects of having low blood sugar,"[effect, low, blood, sugar]"
198,321441,how much is a us postal stamp cost,"[much, us, postal, stamp, cost]"


In [5]:
import pandas as pd
features_test = pd.DataFrame()
for index, query in pipeline.queries_test.iterrows():
    features_test = pd.concat([features_test, pd.DataFrame({
        'qID': [query['qID']] * len(pipeline.collection),
        'pID': pipeline.collection['pID']
    })])